# OpenAI FUnction Calling in Langchain

In [1]:
import os
import openai

In [2]:
from typing import List
from pydantic import BaseModel, Field

In [3]:
class User:
    def __init__(self, name:str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email



In [4]:
foo = User(name="Joe", age=32, email="joe@gmail.com")
foo

In [5]:
foo.name

'Joe'

In [6]:
foo = User(name="Joe", age="herve", email="joe@gmail.com")
foo.age

'herve'

In [7]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [8]:
foo_p = pUser(name='Jane', age=32, email="jane@gmail.com")
foo_p

pUser(name='Jane', age=32, email='jane@gmail.com')

In [9]:
foo_p.name

'Jane'

In [10]:
foo_p = pUser(name='Jane', age="bar", email="jane@gmail.com")

ValidationError: 1 validation error for pUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bar', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/int_parsing

In [11]:
class Class(BaseModel):
    students: List[pUser]

In [12]:
obj = Class(
    students= [pUser(name='Jane', age=18, email="jane@gmail.com"),
               pUser(name='Jane', age=16, email="jane@gmail.com")
               ]
)
obj

Class(students=[pUser(name='Jane', age=18, email='jane@gmail.com'), pUser(name='Jane', age=16, email='jane@gmail.com')])

# Pydantic to OpenAI function definition

In [13]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [16]:
from langchain_core.utils.function_calling import convert_pydantic_to_openai_function

In [15]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

C:\Users\Villard\AppData\Local\Temp\ipykernel_3984\1313296718.py:1: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  weather_function = convert_pydantic_to_openai_function(WeatherSearch)


In [17]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [18]:
from langchain_openai.chat_models import ChatOpenAI

In [19]:
model = ChatOpenAI()
model

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001A15BE682F0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001A15BE698B0>, root_client=<openai.OpenAI object at 0x000001A15AD0F7A0>, root_async_client=<openai.AsyncOpenAI object at 0x000001A15BE68320>, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [20]:
model.invoke("What is the weather in SF today", functions = [weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 69, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-ad6d30b6-9fae-4a36-a397-aca2edd78364-0', usage_metadata={'input_tokens': 69, 'output_tokens': 17, 'total_tokens': 86, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [21]:
model_with_function = model.bind(functions=[weather_function])

In [22]:
model_with_function.invoke("what is the weather is in sf")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 69, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-4ce44d04-eb38-4aa7-b34a-c57d3835b772-0', usage_metadata={'input_tokens': 69, 'output_tokens': 17, 'total_tokens': 86, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [23]:
# force the model to use the function
model_with_forced_function = model.bind(functions=[weather_function], function_call = {"name":"WeatherSearch"})

In [24]:
model_with_forced_function.invoke("what is the weather is in sf")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 79, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-56795c6d-2bdc-4293-b9fc-73540599f99e-0', usage_metadata={'input_tokens': 79, 'output_tokens': 7, 'total_tokens': 86, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [25]:
model_with_forced_function.invoke("Hi")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"JFK"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 73, 'total_tokens': 80, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-aa5cb7fe-d7c6-46eb-9c24-88ccc2b2c07c-0', usage_metadata={'input_tokens': 73, 'output_tokens': 7, 'total_tokens': 80, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# Using in a chain
we use this model bound to function in a chain as we normally would

In [26]:
from langchain.prompts import ChatPromptTemplate

In [28]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpfull assistant"),
    ("user", "{input}")
])

In [29]:
chain = prompt | model_with_function

In [30]:
chain.invoke({"input": "what is the weather in dfw"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"DFW"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 76, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-c7039b2f-60e5-431c-bcad-944b1ed8f3be-0', usage_metadata={'input_tokens': 76, 'output_tokens': 17, 'total_tokens': 93, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# Using Multiple Functions
Even better, we can pass a set of functions and let the LLM decide which to use based on the question content

In [31]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name : str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [32]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch)
]

In [33]:
model_with_functions = model.bind(functions = functions)

In [34]:
model_with_functions.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 116, 'total_tokens': 133, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-e28cd2bb-5ea1-424d-98c4-7b3b48f290b4-0', usage_metadata={'input_tokens': 116, 'output_tokens': 17, 'total_tokens': 133, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [35]:
model_with_functions.invoke("what are three songs by Bruce Melody")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"artist_name":"Bruce Melody","n":3}', 'name': 'ArtistSearch'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 117, 'total_tokens': 139, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-f3fb0eb6-226e-4737-8800-998d62aa7ffa-0', usage_metadata={'input_tokens': 117, 'output_tokens': 22, 'total_tokens': 139, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
model_with_functions.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 110, 'total_tokens': 120, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-696f9bf1-d3dd-4730-8bcf-278b3c062039-0', usage_metadata={'input_tokens': 110, 'output_tokens': 10, 'total_tokens': 120, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

: 